In [1]:
# Modules
import matplotlib.pyplot as plt
import napari
import numpy as np
import os
from math import cos, sin, sqrt
from napari.settings import get_settings
from scipy.optimize import minimize
from skimage import filters
from skimage import io
from skimage.measure import regionprops, label
from skimage.morphology import disk, ball, closing
from sklearn.metrics import mean_squared_error


# Prevents automatic start of interactive event loop
get_settings().application.ipy_interactive = False


# Colorblind-friendly palette
COLOR_PALETTE = ['#e69f00',
                 '#56b4e9',
                 '#009e73',
                 '#f0e442',
                 '#0072b2',
                 '#d55e00',
                 '#cc79a7',
                 '#FFFFFF']

In [2]:
def rotation(angle, coords_a, coords_b):
    # Number of spots
    nn = coords_a.shape[0]
    
    # Initialize an array for the rotated reference (b)
    coords_b_rotated = np.zeros((nn, 2))
    
    # Rotation matrix
    rotation_matrix = np.array([[cos(angle), -sin(angle)],
                                [sin(angle), cos(angle)]])
    
    # Apply rotation (there might be a better algebraic way of computing
    # the dot-product for each vector...)
    for ii in range(0, nn):
        coords_b_rotated[ii,:] = np.dot(rotation_matrix, coords_b[ii,:])
    
    # Calculate the mean squared error from sklearn.metrics
    MSE = mean_squared_error(coords_a, coords_b_rotated)
    
    # Calculate the square root of the  MSE
    # (smoother function, better for optimization)
    RMSE = sqrt(MSE)
    
    # Return the RMSE
    return RMSE

In [3]:
def coords_of_a_in_b(ref_a, ref_b, number, prefix):
    # Empty organized array of coordinates
    coords_a = np.empty((number, 2))
    coords_b = np.empty((number, 2))
    
    # User identification of spots in reference images
    labels_a, user_coords_a = identify_spots(ref_a, number, prefix, size=120)
    labels_b, user_coords_b = identify_spots(ref_b, number, prefix, size=20)

    # Localization of spots in reference images
    loc_coords_a = localize_spots(ref_a, number, close_size=10)
    loc_coords_b = localize_spots(ref_b, number, close_size=5)

    # Connect identified spots with localized ones
    for ii in range(number):
        coords_a[ii, :] = find_closest_spot(user_coords_a[ii, :], loc_coords_a)
        coords_b[ii, :] = find_closest_spot(user_coords_b[ii, :], loc_coords_b)
        
    # Find barycentre of spots
    center_a = coords_a.mean(axis=0)
    center_b = coords_b.mean(axis=0)
    
    # Center spots
    coords_a_centered = coords_a - center_a
    coords_b_centered = coords_b - center_b
    
    # Find norm of sports
    norm_a = np.linalg.norm(coords_a_centered)
    norm_b = np.linalg.norm(coords_b_centered)
    
    # Find scaling factor
    scaling = norm_b / norm_a

    # Scale first reference
    coords_a_scaled = coords_a_centered * scaling
    
    # Find rotation angle
    res = minimize(rotation, 0, args=(coords_a_scaled, coords_b_centered), method="L-BFGS-B")
    rotation_angle = res.x[0]

    # Apply rotation angle to first reference
    rotation_matrix = np.array([[cos(-rotation_angle), -sin(-rotation_angle)],
                                [sin(-rotation_angle), cos(-rotation_angle)]])
    coords_a_rotated = (rotation_matrix@coords_a_scaled.T).T
    
    # Decenter first reference
    coords_a_decentered = coords_a_rotated + center_b
    
    return coords_a_decentered, labels_a

In [4]:
def find_closest_spot(spot, spot_array):
    # Initialize return coordinates
    match = np.empty((1, 2))
    
    # Initialize distance to positive infinity
    distance = float('inf')
    
    # Find closest spot
    for spot_ii in spot_array:
        if squared_distance(spot, spot_ii) < distance:
            match = spot_ii
            distance = squared_distance(spot, spot_ii)
    
    # Closest spot
    closest = np.copy(match)
    
    # Set matched spot in localized matrix to infinity (so it can't be found again)
    match[0] = float('inf')
    match[1] = float('inf')
    
    return closest

In [5]:
def squared_distance(spot_1, spot_2):
    return (spot_2[0]-spot_1[0])**2 + (spot_2[1]-spot_1[1])**2

In [6]:
def localize_spots(ref, number, close_size):
    # Initialize coordinates array
    coords = np.empty((3, 2))
    
    # Apply median filter
    ref_proc = filters.median(ref, disk(1.0))
    
    # Find threshold value
    threshold_value = filters.threshold_otsu(ref)
    
    # Apply threshold
    ref_proc = (ref > threshold_value).astype(int)
    
    # Apply closing operation
    ref_proc = closing(ref_proc, disk(close_size))

    # Apply regionprops
    properties = regionprops(label(ref_proc), ref)
    
    # Counter of spots localized
    localized = 0
    
    # Select valid centroid (area > 100 px)
    for spot in properties:
        if spot.area > 100:
            # Save spot coordinates
            coords[localized,:] = np.asarray(spot.weighted_centroid)
            
            # Increase counter
            localized += 1
    
    # Check that all the spots have been localized
    if localized != number:
        print('ERROR: Not enough spots located in reference image!')
    
    return coords

In [7]:
def identify_spots(ref, number, prefix, size):
    # Initialize spot coordinates array
    points = 50 * np.ones((number, 2))
    
    # Select colors from palette
    colors = COLOR_PALETTE[:number]
    
    # Initialize spot labels
    labels = []
    
    # Loop over the number of spots
    for ii in range(number):
        # Initialize annotations in top-left corner
        points[ii, 0] = ii * 2 * size + 2 * size
        
        # Add labels to annotations
        labels.append(prefix + str(ii+1))
    
    # Create text for annotations
    text = {'string': labels,
            'size': 6,
            'color': 'white',
            'translation': 0.75 * np.array([-size, size])}    

    # Open Napari viewer
    viewer = napari.Viewer()
    
    # Add reference image in a new layer
    ref_layer = viewer.add_image(ref, name='Reference image')
    
    # Update colormap
    ref_layer.colormap = 'viridis'
    
    # Add annotations to points layer    
    pt_layer = viewer.add_points(points,
                                 properties={'label': labels},
                                 text=text,
                                 size=size,
                                 edge_width=.05,
                                 edge_color=colors,
                                 face_color='transparent',
                                 name='ROIs')

    # Change to select mode
    pt_layer.mode = 'select'
    
    # Close viewer and retrieve spot coordinates
    @viewer.bind_key('Enter')
    def complete_spot_selection(viewer):
        viewer.close()
        
    # Run Napari
    napari.run()
    
    # Return sport labels and coordinates
    return pt_layer.properties['label'], pt_layer.data

In [12]:
def main():
    # Folder
    folder = r'E:\PythonScripts\3i_Z_Stack'
    
    # Reference images
    ref_a = np.asarray(io.imread(os.path.join(folder, 'up_ref.tif')))
    temp_b = np.asarray(io.imread(os.path.join(folder, 'sub_ref.tif')))
    
    # Crop reference b
    (size_x, size_y) = temp_b.shape
    cust_size = 100
    ref_b = temp_b[int(size_x/2)-100:int(size_x/2)+100, int(size_y/2)-100:int(size_y/2)+100]
    
    # Number of spots
    number = 3
    
    # Color palette
    colors = COLOR_PALETTE[:number]
    
    # Label prefix
    prefix = 'S'
    
    # Find transform
    a_in_b, labels = coords_of_a_in_b(ref_a, ref_b, number, prefix)
    
    # Display transform
    viewer = napari.Viewer()
    ref_layer = viewer.add_image(ref_b, name='Reference image')
    ref_layer.colormap = 'viridis'
    text = {'string': labels.tolist(),
            'size': 6,
            'color': 'white',
            'translation': 0.75 * np.array([-20, 20])}
    annotation_layer = viewer.add_points(a_in_b,
                                         properties={'label': labels},
                                         text=text,
                                         size=20,
                                         edge_width=.05,
                                         edge_color=colors,
                                         face_color='transparent',
                                         name='ROIs')
    napari.run()
    
    # Stack
    stack = io.imread(os.path.join(folder, 'stack.tif'))

    # Crop stack
    crop = stack[:, int(size_x/2)-100:int(size_x/2)+100, int(size_y/2)-100:int(size_y/2)+100]
    
    # Apply median filter to crop
    crop = filters.median(crop, ball(1))
    
    # Open Napari viewer
    viewer = napari.Viewer()
    
    # Add stack in new layer
    stack_layer = viewer.add_image(crop, name='Stack')
    
    # Update colormap
    stack_layer.colormap = 'viridis'
    
    # Z coordinates
    z_column = np.empty((3, 1))
    z_column[0] = 50
    z_column[1] = 20
    z_column[2] = 80
    
    # Coordinates of annotations in 3D
    points = np.concatenate((z_column, a_in_b), axis=1)
    
    # Add annotations in new layer
    viewer.dims.ndisplay = 3
    text = {'string': labels.tolist(),
            'size': 16,
            'color': 'white',
            'anchor': 'upper_right',
            'translation': np.array([0, 10, 10])}
    annotation_layer = viewer.add_points(points,
                                         properties={'label': labels},
                                         face_color=colors,
                                         size=10,
                                         shading='spherical',
                                         edge_width=0,
                                         opacity=0.6)
    text_layer = viewer.add_points(points,
                                   name='Text',
                                   text=text,
                                   face_color='transparent',
                                   edge_width=0)
    
    # Run Napari
    napari.run()
    
    
if __name__ == '__main__':
    main()

C:\Users\David Nguyen\anaconda3\envs\Hologram_3i_check\lib\site-packages\napari\_qt\widgets\qt_dims.py:294: UserWarning: Refusing to play a hidden axis!
